In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [2]:
gdd.download_file_from_google_drive(
    file_id='1nPeBbrTQH_fdvh9prmshA8fu3KrZTSSU',
    dest_path='data/TIMIT.zip'
)

In [3]:
!unzip data/TIMIT.zip

Archive:  data/TIMIT.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data/TIMIT.zip or
        data/TIMIT.zip.zip, and cannot find data/TIMIT.zip.ZIP, period.


In [4]:
gdd.download_file_from_google_drive(
    file_id='1DMUmji3R96J4yM7F6k1ov8XJ4nZTFjDH',
    dest_path='data/SHRUTI.zip'
)

In [5]:
!unzip data/SHRUTI.zip

Archive:  data/SHRUTI.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data/SHRUTI.zip or
        data/SHRUTI.zip.zip, and cannot find data/SHRUTI.zip.ZIP, period.


In [6]:
gdd.download_file_from_google_drive(
    file_id='1Gb0lEZUxqYBHuDaDpLkwd-6x420zdWvE',
    dest_path='data/khanty_4.zip'
)

In [7]:
!unzip data/khanty_4.zip -d khanty_4

Archive:  data/khanty_4.zip
  inflating: khanty_4/F1-0.wav       
  inflating: khanty_4/__MACOSX/._F1-0.wav  
  inflating: khanty_4/F1-1.wav       
  inflating: khanty_4/F1-2.wav       
  inflating: khanty_4/F1-3.wav       
  inflating: khanty_4/F1-4.wav       
  inflating: khanty_4/F1-5.wav       
  inflating: khanty_4/F1-6.wav       
  inflating: khanty_4/F1-7.wav       
  inflating: khanty_4/F1-8.wav       
  inflating: khanty_4/F1-9.wav       
  inflating: khanty_4/F1-10.wav      
  inflating: khanty_4/F1-11.wav      
  inflating: khanty_4/F2-0.wav       
  inflating: khanty_4/F2-1.wav       
  inflating: khanty_4/F2-2.wav       
  inflating: khanty_4/F2-3.wav       
  inflating: khanty_4/F2-4.wav       
  inflating: khanty_4/F2-5.wav       
  inflating: khanty_4/F2-6.wav       
  inflating: khanty_4/F2-7.wav       
  inflating: khanty_4/F2-8.wav       
  inflating: khanty_4/F2-9.wav       
  inflating: khanty_4/F2-10.wav      
  inflating: khanty_4/F2-11.wav      
  inflating: kha

In [8]:
!pip install tensorly

     |████████████████████████████████| 112kB 8.3MB/s 
     |████████████████████████████████| 163kB 13.2MB/s 
  Created wheel for tensorly: filename=tensorly-0.5.1-cp36-none-any.whl size=149168 sha256=1d9cedfd95bb56163290e211c43446cdff5af45e65de7359d38c0f8a3bbde62b
  Stored in directory: /root/.cache/pip/wheels/88/1e/e7/b9677b2046cc87e17931b4b5781941786d3ee647825ca40ea6
Successfully built tensorly


In [9]:
import glob
import os
from os.path import join

import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorly as tl
from tensorly.decomposition import symmetric_parafac_power_iteration

In [19]:
def get_features_filtered_by_label(features, labels, selected_label):
    return [feature for feature, label in zip(features, labels) if label == selected_label]

# check normalize
def get_k_main_eigenvectors_svd(features, k=1):
    _, _, vh = np.linalg.svd(normalize(features), full_matrices=False)
    return vh[:k]

def get_k_main_eigenvectors(X, k=1):
    m1 = X.mean(axis=0)
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0])
    w, v = np.linalg.eigh(m2)
    return v[:, -k:].T

def whiten(m2, eps=1e-18):
   # eigenvalue decomposition of the covariance matrix
   d, V = np.linalg.eigh(m2)

   # a fudge factor can be used so that eigenvectors associated with
   # small eigenvalues do not get overamplified.
   D = np.diag(1. / np.sqrt(d+eps))

   # whitening matrix
   W = np.dot(V, D)

   return W

def get_k_main_eigenvectors_cpd(X, k=1):
    m1 = X.mean(axis=0)
    N = X.shape[1]
    corr_matrix = np.cov(X.T)
    corr_eigens = np.linalg.eigvalsh(corr_matrix)
    m2 = corr_matrix + np.outer(m1, m1) - corr_eigens[0] * np.eye(corr_matrix.shape[0], dtype='float64')
    m3 = np.einsum('ij,ik,il->jkl',X,X,X) / N
    basis_sum = np.einsum('ij,ik,il->jkl',  m1[None, :], np.eye(N), np.eye(N)) + \
            np.einsum('ij,ik,il->jkl', np.eye(N),  m1[None, :], np.eye(N)) + \
            np.einsum('ij,ik,il->jkl', np.eye(N), np.eye(N),  m1[None, :])
    # basis_sum = np.einsum('j,ik,il->jkl',  m1, np.eye(N), np.eye(N)) + \
    #         np.einsum('ij,k,il->jkl', np.eye(N),  m1, np.eye(N)) + \
    #         np.einsum('ij,ik,l->jkl', np.eye(N), np.eye(N),  m1)
    m3 = m3 - corr_eigens[0] * basis_sum
    w = whiten(m2)
    # m3_whiten = tl.tucker_to_tensor((m3, [w, w, w]))
    m3_whiten = np.einsum('nmp,nk,ml,pd->kld', m3, w, w, w)
    lambdas, V = symmetric_parafac_power_iteration(m3_whiten, rank=k, n_iteration=20)
    A = np.linalg.pinv(w.T) @ V @ np.diag(lambdas)
    return A.T # V.T

def get_name(path):
    return os.path.splitext(os.path.split(path)[1])[0]

def test(eigen_female, eigen_male, features_test, is_correct_function, debug=False):
    counter = 0
    for feature in features_test:
        eigen_test = get_k_main_eigenvectors(feature, k=num_eigenvectors)
        dist_to_female = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_female, eigen_test)])
        dist_to_male = min([np.square(ete - etr).sum() for etr, ete in zip(eigen_male, eigen_test)])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

def test_unnormmse(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    for feature in features_test:
        dist_to_female = sum([min([np.linalg.norm(x - etr, ord=2)
                                   for etr in eigen_female]) for x in feature])
        dist_to_male = sum([min([np.linalg.norm(x - etr, ord=2)
                                   for etr in eigen_male]) for x in feature])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

def test_cosine(eigen_female, eigen_male, features_test, is_correct_function, debug=False, **kwargs):
    counter = 0
    for feature in features_test:
        dist_to_female = sum([min([1 - np.square(np.dot(x, etr)) / (np.dot(x, x) * np.dot(etr, etr))
                                   for etr in eigen_female]) for x in feature])
        dist_to_male = sum([min([1 - np.square(np.dot(x, etr)) / (np.dot(x, x) * np.dot(etr, etr))
                                   for etr in eigen_male]) for x in feature])
        correct = is_correct_function(dist_to_female, dist_to_male)
        if correct:
            counter += 1
    if debug:
        print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    return counter / len(features_test)

# -----------------------------------------------------------------
# def normalize(x):
#     return x - x.mean()

def normalize(x):
    return x / np.linalg.norm(x, ord=2, axis=1, keepdims=True)

In [12]:
paths = sorted(glob.glob('khanty_4/*.wav'))

# First variant: 
1. split train/test for whole files (not single vectors)
2. calculate SVD for both train and test

In [13]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:48<00:00,  3.09it/s]


In [14]:
X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list,
                                                    test_size=0.33, random_state=120, stratify=labels_list)

In [20]:
for random_state in [120, 5544, 33, 23, 997, 719, 581]:
    X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list, test_size=0.33,
                                                        random_state=random_state, stratify=labels_list)

    num_eigenvectors = 4 # num_eigenvectors > 1 doesn't improve accuracy; need to improve inference

    features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
    eigen_female = get_k_main_eigenvectors_cpd(features, k=num_eigenvectors)

    features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
    eigen_male = get_k_main_eigenvectors_cpd(features, k=num_eigenvectors)

    features_test = get_features_filtered_by_label(X_test, y_test, 'F')
    is_correct_function = lambda x, y: x < y
    accuracy_female = test_cosine(eigen_female, eigen_male, features_test, is_correct_function)

    features_test = get_features_filtered_by_label(X_test, y_test, 'M')
    is_correct_function = lambda x, y: x > y
    accuracy_male = test_cosine(eigen_female, eigen_male, features_test, is_correct_function)

    print(f'f: {accuracy_female:.3f}, m: {accuracy_male:.3f}')

f: 0.873, m: 0.871
f: 0.853, m: 0.957
f: 0.931, m: 0.814
f: 0.804, m: 0.871
f: 0.882, m: 0.857
f: 0.902, m: 0.886
f: 0.863, m: 0.857
